In [1]:
import cv2
from matplotlib import pyplot as plt
#import osp
#import imutils
import numpy as np

name='rho'
print('true')

true


In [2]:
def rect(c):
    area = cv2.contourArea(c)
    print('contour area=', area)
    app  = cv2.approxPolyDP(c, 0.0009 * cv2.arcLength(c, True), True)
    (p,q,r,s)= cv2.boundingRect(app)
    
    # compute the aspect ratio of the contour, which is simply the width
    # divided by the height of the bounding box
#     aspectRatio = w / float(h)
#     print('aspect ratio of'+ name,'=',aspectRatio)
#     # use the area of the contour and the bounding box area to compute
#     # the extent
#     extent = area / float(w * h)
#  
    # compute the convex hull of the contour, then use the area of the
    # original contour and the area of the convex hull to compute the
    # solidity
    hull = cv2.convexHull(app)
    (x, y, w, h) = cv2.boundingRect(hull)
    roi = resized[x:x+w,y:y+h]
    cv2.imshow('roi', roi)
    hullArea = cv2.contourArea(hull)
    print('hull area of',name, hullArea)
    solidity = area / float(hullArea)
    print('solidity of'+ name,'=',solidity)
 
    # visualize the original contours and the convex hull and initialize
    # the name of the shape
#    x,y,w,h=p,q,r,s
    print('p=',p)
    print('q=',q)
    print('r=',r)
    print('s=',s)
    cv2.putText(resized, 'p', (p, p), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 2)
    cv2.putText(resized, 'q', (q, q), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 2)
    cv2.putText(resized, 'r', (r, p), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 2)
    cv2.putText(resized, 's', (q, s), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 2)
    
    cv2.line(resized,(p,p),(q+r,q+s),(0,0,0),4)
    cv2.line(resized,(p+s,p+s),(p+r,p+s),(180,0,0),4)
    
    #cv2.drawContours(hullImage, [hull], -1, 255, -1)
    cv2.line(hullImage,(x,w-y),(x+w,w-x),(255,255,255),3)
    cv2.line(hullImage,(x,w),(y,w),(255,255,255),5)
    #line_x = cv2.line(resized,(s-p,s-q),(2*s+p,s-q),(0,0,255),5)
    #line_y=cv2.line(resized,(s-q,s-q),(s-q+p,s-p+r),(189,40,150),3)
#     line_x=cv2.line(resized,(q,r-p),(p,s),(255,0,150),3)
    #print('angle=',np.mean(np.array(np.arctan(np.divide(line_y,line_x))[0])))
    #cv2.drawContours(resized, [app], -1, (255, 255, 0), 3)
    #cv2.imshow("Convex Hull", hullImage)
    

In [3]:
def detect(c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.009 * peri, True)
#         view(approx)
        if len(approx) == 3:
            shape = "triangle"
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            rect(c)
            # compute the bounding box of the contour and use the
#             # bounding box to compute the aspect ratio
#             (x, y, w, h) = cv2.boundingRect(approx)
#             ar = w / float(h)
#             print('ar of',name,'=',ar)
#             # a square will have an aspect ratio that is approximately
#             # equal to one, otherwise, the shape is a rectangle
#             shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
#         # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
        elif len(approx) == 6:
            shape = "hexagon"
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
        # return the name of the shape
        return shape


In [4]:
def show(img):
    cv2.imshow(str(img), img)
    cv2.waitKey(0)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()


In [5]:
image = cv2.imread("Samples/"+name+".png")
scale_percent = 100 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(image, dim)
ratio = image.shape[0] / float(resized.shape[0])
#show(resized)

AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
hullImage = np.zeros(resized.shape[:2], dtype="uint8")
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 140, 255, cv2.THRESH_BINARY_INV)[1]
#cv2.imshow('thresh',thresh)
# find contours in the thresholded image and initialize the
# shape detector
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
elif len(cnts) == 3:
        cnts = cnts[1]
#show(thresh)

NameError: name 'resized' is not defined

In [7]:
for c in cnts :
    M = cv2.moments(c)
    cX = int((M["m10"] / M["m00"]) * ratio)
    cY = int((M["m01"] / M["m00"]) * ratio)
    shape = detect(c)
    
    # multiply the contour (x, y)-coordinates by the resize ratio,
    # then draw the contours and the name of the shape on the image
    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    print(shape, cv2.arcLength(c, True),cX, cY)
    #cv2.drawContours(resized, [c], -1, (0, 255, 0), 5)
    cv2.putText(resized, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, (0, 0, 0), 2)
show(resized)

NameError: name 'cnts' is not defined